First, the image shows up, and the user should form an initial contour around it by clicking on arbitrary spots around the main object (which is a Tasbih in this image).
Then, the main algorithm begins, and the contour shrinks over several iterations until it fits into the object's external borders.
In each iteration, the corresponding image should be saved. Then, by putting all these images together, we can observe the contour's changing process.
The line associated with saving each iteration's result has been commented. But the final result of merging those images into an .mp4 file is included.

تصویر یک بار به صورت عادی و یک بار به صورت سیاه و سفید خوانده شده و در متغیر های مربوطه قرار می گیرد

In [34]:
import os
import cv2
import numpy as np
from math import sqrt
tasbih = cv2.imread( os.getcwd()+'\\'+'tasbih.jpg',cv2.IMREAD_UNCHANGED)
tasbih_bw = cv2.imread( os.getcwd()+'\\'+'tasbih.jpg',cv2.IMREAD_GRAYSCALE)

تابع زیر کلیک های کاربر روی تصویر نمایش داده شده ی تسبیح را تشخیص داده و هر بار نقطه جدید را با یک خط به نقطه قبلی وصل می کند
در نهایت کاربر باید چندضلعی را به گونه ای رسم کند که آخرین نقطه ی چند ضلعی تقریبا منطق بر نقطه آغازین باشد
تابع به طور خودکار آخرین نقطه را حذف می کند و نقاط اولیه ی کانتور را در متغیر گلوبالی قرار می دهد

In [35]:

def mouse_listener_tasbih(event, x, y, flags, param):
   global draw_points
   if event == cv2.EVENT_LBUTTONDOWN:
    #    print(x,y)
       draw_points.append(np.array([[x,y]]))
       correct_points.append(np.array([[y,x]]))
       help = np.array(draw_points,np.int32)
       help = help.reshape((-1,1,2))
       cv2.polylines(tasbih,[help] ,False,(250,250,250), thickness = 2)
       cv2.imshow("tasbih",tasbih)
    #    print(points)
    
draw_points = []
correct_points = []
cv2.namedWindow("tasbih")
cv2.setMouseCallback("tasbih", mouse_listener_tasbih)

# cv2.destroyAllWindows()
cv2.imshow("tasbih",tasbih)
cv2.waitKey(0)
cv2.destroyAllWindows()
# print(points)
points = correct_points[:-1]
# print(points)


530 155
540 159
535 170
548 168
532 198
547 187
547 211
556 209
550 232
567 218
584 233
593 222
598 239
609 224
616 238
632 217
648 233
661 213
671 227
684 210
697 222
710 204
718 218
727 209
734 222
745 201
760 214
781 181
797 201
806 180
815 195
825 182
831 202
847 180
851 293
864 277
877 257
877 279
894 254
904 263
910 250
919 271
937 255
937 271
957 267
952 280
972 290
957 294
986 305
974 313
1008 323
988 330
1007 338
988 338
1009 354
987 352
1017 367
985 363
1000 377
956 375
938 388
922 371
913 369
918 388
896 380
896 395
866 375
862 391
842 377
838 405
809 396
822 428
809 431
823 440
788 437
797 449
766 451
777 459
753 457
761 469
747 467
748 480
730 468
730 488
716 477
716 494
703 483
704 503
684 491
685 509
665 490
663 509
653 501
652 516
642 508
641 524
626 513
629 535
616 523
607 548
590 537
594 550
585 543
584 560
568 549
566 564
551 556
552 569
541 556
539 574
524 568
525 580
508 565
511 585
503 582
502 598
487 595
485 620
480 614
477 628
461 616
455 638
440 622
437 639
423

در اینجا گرادیان تصویر سیاه و سفید محاسبه شده و درواقع ماتریسی به دست می آید که مجموع مربعات گرادیان افقی و عمودی ضربدر مقدار ثابت گاما است و در آینده قرار است برای محاسبه انرژی اکسترنال کانتور از مقادیری که قبلا در ماتریس گرادیان قرار گرفته اند استفاده شود

In [36]:

gradian = cv2.Canny(tasbih_bw,110,190).astype(float)
cv2.imshow("tasbih",gradian)
cv2.waitKey(0)
cv2.destroyAllWindows()
gamma = 5
gradian = -gamma*(gradian*gradian)

برای محاسبه کم ترین انرژی ممکن برای کانتور، وقتی هر یک از نقاط کانتور در یک همسایگی سه در سه جا به جا می شوند،‌ از ایده داینامیک پروگرمینگ برای بهینه شدن زمان محاسبه کمترین انرژی استفاده می کردیم
تابع زیر،‌ با گرفتن یک ستون از کمترین انرژی های یک راس در ۹ نقطه ی مختلف از همسایگی و همچنین مکان نقطه ی بعدی کانتور و بقیه ی پارامتر های لازم برای محاسبه کمترین انرژی، کمترین انرژی لازم برای رسیدن به نقطه ی موردنظر در جایگاه خاصی از همسایگی سه در سه اش را محاسبه می کند

In [37]:
def calculate(arr_1,i_1,j_2,col_2,d,center_mean):
      
    point_i1 = adj_finder(contour_points[col_2-1], i_1)
    if col_2 == n:
        col_2 = 0
    point_i2 = adj_finder(contour_points[col_2], j_2)

    cnt_x_mean = center_mean[0]
    cnt_y_mean = center_mean[1]
    beta = 1000
    dis_from_center = beta*((sqrt((cnt_x_mean-point_i1[0][0])**2 + (cnt_y_mean-point_i1[0][1])**2) + sqrt((cnt_x_mean-point_i2[0][0])**2 + (cnt_y_mean-point_i2[0][1])**2 )))

    alpha = 5
    # d = 0 
    res = arr_1 + gradian[point_i1[0][0]][point_i1[0][1]]+ gradian[point_i2[0][0]][point_i2[0][1]] + alpha*((point_i1[0][0]-point_i2[0][0])**2 + (point_i1[0][1]-point_i2[0][1])**2 - d**2) +  dis_from_center
    
    # print(res)
    return res

هر بار که نقاط کانتور عوض می شوند، پارامتر 

In [38]:
def calculate_d():
    global contour_points
    # print(contour_points)
    help1 = np.zeros(contour_points.shape, dtype= 'uint16')
    # print(contour_points[:-1,:,:])
    help1[:-1,:,:] = contour_points[1:,:,:]
    # print(help1)
    help1[contour_points.shape[0]-1,:,:] =  contour_points[0,:,:]
    # print(help1)
    # print(contour_points)
    square_of_dists = ((contour_points-help1)**2)

    sqr_x_es = square_of_dists[:,:,0]
    sqr_y_es = square_of_dists[:,:,1]
    sum_xy = np.sqrt(sqr_x_es + sqr_y_es)
    sums = np.sum(sum_xy)
    d = sums/contour_points.shape[0]
    return d 
    # print(contour_points)

تابع محاسبه نقاط جدید کانتور

In [39]:
def update_contour(bichare,index):
    global contour_points
    global n
    i = n
    current_contour_points = []
    current_contour_points.append(adj_finder(contour_points[0], index))
    # index = bichare[i][index]
    index = bichare[index][i]

    while i > 1:
        i-=1
        current_contour_points.append(adj_finder(contour_points[i], index))
        # print(bichare)
        # index = bichare[i][index]
        index = bichare[index][i]

    # snapshot(current_contour_points)
    # print(current_contour_points)
    current_contour_points = np.array(current_contour_points[:])
    # print(current_contour_points.shape)
    return current_contour_points

تابعی که از روی شماره ی سطر، مختصات نقطه ی متناظر در همسایگی نقطه را می دهد

In [40]:
def adj_finder(point, index):
    x = 0
    y = 0
    if index<3:
        y-=1
    if index > 5:
        y+=1
    if index == 0 or index == 3 or index == 6:
        x-=1
    if index == 2 or index == 5 or index == 8:
        x+=1
    return np.array([[x+point[0][0],y+point[0][1]]])

قسمت اصلی مربوط به آپدیت کانتور و تغییرات آن

In [41]:
n = len(points)

contour_points = np.array(points[:])
contour_val = 0 

itr_times = 600
d = calculate_d()
prev_d = d
darsad = 0.99
itr = 0
mean = np.array([457 ,362])
while itr<itr_times and prev_d>=darsad*d:
    # mean = np.array([np.mean(contour_points[:,0,0]), np.mean(contour_points[:,0,1])])
    itr+=1
    bichare = np.zeros((9,n+1),dtype=np.int)
    index = 0 
    for i in range(0,9):
        arr1 = np.zeros((9),dtype = np.float32)
        arr2 = np.zeros((9),dtype= np.float32)

        # for ii in range(0,9):
        #     arr2[ii] = calculate(0,i,ii,1,d)

        # current_contour_points.append([coun])
   
        for col in range(1,n):
            arr2 = np.zeros((9),dtype= np.float32)
            for j in range(0,9):
                for k in range(0,9):
                    temp =  calculate(arr1[k],k,j,col,d, mean)
                    if arr2[j] > temp or k==0:
                        arr2[j] = temp
                        # bichare[col][j] = k
                        bichare[j][col] = k
                        # print(bichare)
                        # print(bichare[j][col])
            # current_contour_points.append(best_k)

            arr1[:] = arr2[:]
        arr2 = np.zeros((9),dtype= np.float32)
        for ii in range(0,9):
            temp = calculate(arr1[ii],ii,i,n,d, mean)
            if arr2[i] > temp or ii==0:
                arr2[i] = temp
                # bichare[n][i] = ii
                bichare[i][n] = ii


        if i==0 or contour_val > arr2[i]:
            contour_val = arr2[i]
            saved_bichare = bichare[:,:]
            index = i
        # current_contour_points = 
        # current_contour_points[:] = current_contour_points[:]

        #update contour_points
    contour_points = update_contour(saved_bichare,index)[:,:,:]
        #caculate d please
    prev_d = d
    d = calculate_d()
    # reverse = np.zeros(contour_points.shape,np.int32)
    # reverse[:,:,1] = contour_points[:,:,0]
    # reverse[:,:,0] = contour_points[:,:,1]
    # help = np.array(reverse,np.int32)
    # help = help.reshape((-1,1,2))
    # tasbih = cv2.imread('tasbih.jpg',cv2.IMREAD_UNCHANGED)
    # cv2.polylines(tasbih,[help] ,True,(250,250,250), thickness = 2)
    # cv2.imwrite(str(itr).zfill(5)+'.jpg', tasbih)
reverse = np.zeros(contour_points.shape,np.int32)
reverse[:,:,1] = contour_points[:,:,0]
reverse[:,:,0] = contour_points[:,:,1]
help = np.array(reverse,np.int32)
help = help.reshape((-1,1,2))
tasbih = cv2.imread('tasbih.jpg',cv2.IMREAD_UNCHANGED)
cv2.polylines(tasbih,[help] ,True,(250,250,250), thickness = 2)
cv2.imwrite('final_contour.jpg', tasbih)
